In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
# from selenium import webdriver

no_of_pages = 1
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

def jobs_extract(job_list):
    title = []
    company = []
    location = []
    days_since_posted = []
    link = []
    career_level = []
    qualification = []
    years_of_experience = []
    job_type = []
    specialization = []
    sql = []
    excel = []
    tableau = []
    python = []
    extracted_info = []
    
    for job_post in job_list:
        # Job Title
        print(job_post.find('div', attrs={'class': 'z1s6m00 l3gun70 l3gun74 l3gun72'}).text)
        title.append(job_post.find('div', attrs={'class': 'z1s6m00 l3gun70 l3gun74 l3gun72'}).text)

        # Company Name
#         company = job_post.find('span', attrs={'class': 'z1s6m00 _17dyj7u1 _1hbhsw64u _1hbhsw60 _1hbhsw6r'}).text
        company.append(job_post.find('a', attrs={'data-automation': 'jobCardCompanyLink'}).text)

        # Location
#         location = job_post.find('a', attrs={'data-automation': 'jobCardLocationLink'}).text
        location.append(job_post.find('span', attrs={'class': 'z1s6m00 _1hbhsw64u y44q7i0 y44q7i3 y44q7i21 y44q7ih'}).text)

        # Days since posted
        days_since_posted.append(job_post.find('time', attrs={'class': 'z1s6m00 _1hbhsw64u'}).text)
        
        # Full details
        details = job_post.find('a', attrs={'class': 'jdlu994 jdlu996 jdlu999 y44q7i2 z1s6m00 z1s6m0f _1hbhsw6h'}, href=True)

        job_post_url = 'https://www.jobstreet.com.ph' + str(details['href'])
        job_post_page = requests.get(job_post_url, headers=headers)
        link.append(job_post_url)
        
        job_post_soup1 = BeautifulSoup(job_post_page.content,"html.parser")
        job_post_soup2 = BeautifulSoup(job_post_soup1.prettify(), "html.parser")

        job_post_div = job_post_soup2.findAll('div', attrs={'data-automation': 'jobDescription'})
        
        for job_desc in job_post_div:
            skills = job_desc.text

            if "sql" in skills.lower():
                sql.append("Yes")
            else:
                sql.append("No")
                
            if "excel" in skills.lower():
                excel.append("Yes")
            else:
                excel.append("No")
                
            if "tableau" in skills.lower():
                tableau.append("Yes")
            else:
                tableau.append("No")
                
            if "python" in skills.lower():
                python.append("Yes")   
            else:
                python.append("No")
        
        # Additional job information
        job_post_div2 = job_post_soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw6r pmwfa50 pmwfa57'})
        
        for job_addtl_desc in job_post_div2:
            skills2 = job_addtl_desc.text
#             print ("\nSKILL: " + skills2)
            
            #Add ignore case
            if "career level" in skills2.lower():
                career_level.append(skills2.replace("Career Level", "").strip())                
            elif "qualification" in skills2.lower():
                qualification.append(skills2.replace("Qualification", "").strip())                
            elif "years of experience" in skills2.lower():
                years_of_experience.append(skills2.replace("Years of Experience", "").strip())
            elif "job type" in skills2.lower():
                job_type.append(skills2.replace("Job Type", "").strip())                
            elif "job specializations" in skills2.lower():
                specialization.append(skills2.replace("Job Specializations", "").strip())
                   
    #Save all information
    extracted_info.append(title)
    extracted_info.append(company)
    extracted_info.append(days_since_posted)
    extracted_info.append(link)
    extracted_info.append(career_level)
    extracted_info.append(qualification)
#     extracted_info.append(years_of_experience)
#     extracted_info.append(job_type)
    extracted_info.append(specialization)
    extracted_info.append(sql)
    extracted_info.append(excel)
    extracted_info.append(tableau)
    extracted_info.append(python)
    
#     cols = ['title', 'company', 'days_since_posted', 'link', 'career_level', 'qualification',
#             'years_of_experience', 'job_type', 'specialization', 'sql', 'excel', 'tableau', 'python']
    cols = ['title', 'company', 'days_since_posted', 'link', 'career_level', 'qualification',
            'specialization', 'sql', 'excel', 'tableau', 'python']
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    print("lengths: \n")
    print(len(title))
    print(len(company))
    print(len(days_since_posted))
    print(len(link))
    print(len(career_level))
    print(len(qualification))
    print(len(years_of_experience))
    print(len(job_type))
    print(len(specialization))
    print(len(sql))
    print(len(excel))
    print(len(tableau))
    print(len(python))
    
    return jobs_list
    
def jobs_save(all_job_list):
    filename = "jobstreet_dataset.xls"
    jobs = pd.DataFrame(all_job_list)
    jobs.to_excel(filename)    
                                        
def jobs_search():
    for page in range(no_of_pages):
        # url = 'https://www.indeed.com.ph/jobs?q=' + skill + '&l=' + place + '&sort=date&start=' + str(page * 10)
        url = 'https://www.jobstreet.com.ph/en/job-search/data-analyst-jobs-in-philippines/' + str(page)
        page = requests.get(url, headers=headers)
        soup1 = BeautifulSoup(page.content,"html.parser")
        soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

        job_list = soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw69e _1hbhsw68e _1hbhsw672 _1hbhsw67e'})
#         job_list = soup2.findAll('div', attrs={'data-automation': 'jobListing'})
        
        all_job_list = jobs_extract(job_list)
        jobs_save(all_job_list)
                
jobs_search()